# Priority Search Queue
###### (https://www.cs.ox.ac.uk/ralf.hinze/publications/ICFP01.pdf)

Priority search queues combine the searchability of keys within a balanced binary tree with a heap invariant over each node's priority.

We're going to use the outcome of a tournament as an example of how to build a PSQ.

In [2]:
data PSQ k p = Void
             | Winner (k, p) -- ^ The winner's key and priority
                      (LTree k p) -- ^ The tree of losers
                      k -- ^ the maximum key (used for merging)

type Size = Int

data LTree k p = Start
               | Loser !Size
                       (k, p) -- ^ A loser's key and priority
                       (LTree k p) -- ^ Left subtree
                       k -- ^ the 'split key'
                       (LTree k p) -- ^ Right subtree
                       
-- If the player's priority is less than or equal to the split key then they dominated the left subtree,
-- otherwise they dominated the right subtree.

key = fst
prio = snd

maxKey :: PSQ k p -> k
maxKey (Winner _ _ m) = m

size :: LTree k p -> Size
size Start = 0
size (Loser s _ _ _ _) = s

empty :: PSQ k p
empty = Void

singleton :: (k, p) -> PSQ k p
singleton b = Winner b Start (key b)

node :: (k, p) -> LTree k p -> k -> LTree k p -> LTree k p
node b l k r = Loser (size l + 1 + size r)
                     b l k r

-- Merge two queues with the precondition that all keys in the left one are strictly smaller than the keys in the second.
merge :: Ord p => PSQ k p -> PSQ k p -> PSQ k p
merge Void p = p
merge p Void = p
merge (Winner b t m) (Winner b' t' m')
  | prio b <= prio b' = Winner b (node b' t m t') m'
  | otherwise         = Winner b' (node b t m t') m'
  
fromOrdList :: Ord p => [(k, p)] -> PSQ k p
fromOrdList = head . foldm empty . map singleton where
  foldm acc [] = [acc]
  foldm _ xs = until single group xs
  single [_] = True
  single _ = False
  group (x:y:xs) = x `merge` y : group xs
  group xs = xs
  
popWinner :: (Ord p, Ord k) => PSQ k p -> Maybe ((k, p), PSQ k p)
popWinner Void = Nothing
popWinner (Winner b t m) = Just (b, secondBest t m)

-- Replay the tournament without the champion to get the second best player
secondBest :: (Ord p, Ord k) => LTree k p -> k -> PSQ k p
secondBest Start m = Void
secondBest (Loser _ b t k u) m
  | key b <= k = Winner b t k `merge` secondBest u m
  | otherwise = secondBest t k `merge` Winner b u m
  
toOrdList :: (Ord k) => PSQ k p -> [(k, p)]
toOrdList Void = []
toOrdList (Winner b t m) = ($ []) $ trav b t where
  trav b Start = (b :)
  trav b (Loser _ b' tl k tr)
    | key b' <= k = trav b' tl . trav b tr
    | otherwise   = trav b tl . trav b' tr
    
data TreeView k p
  = Z
  | S (k, p)
  | M (PSQ k p) (PSQ k p) -- all keys in left are strictly less than those in right
    
treeView :: Ord k => PSQ k p -> TreeView k p
treeView Void = Z
treeView (Winner b Start _) = S b
treeView (Winner b (Loser _ b' tl sk tr) m)
  | key b' <= sk = M (Winner b' tl sk) (Winner b tr m)
  | otherwise    = M (Winner b tl sk) (Winner b' tr m)

lookup :: Ord k => k -> PSQ k p -> Maybe p
lookup k psq = case treeView psq of
  Z -> Nothing
  S b -> if k == key b then Just (prio b) else Nothing
  M tl tr
    | k <= maxKey tl -> lookup k tl
    | otherwise -> lookup k tr

adjust :: (Ord k, Ord p) => (p -> p) -> k -> PSQ k p -> PSQ k p
adjust f k psq = case treeView psq of
  Z -> Void
  S b -> if k == key b then singleton (k, f $ prio b)
                       else singleton b
  M tl tr
    | k <= maxKey tl -> adjust f k tl `merge` tr
    | otherwise -> tl `merge` adjust f k tr
    
insert :: (Ord k, Ord p) => (k, p) -> PSQ k p -> PSQ k p
insert b psq = case treeView psq of
  Z -> singleton b
  S b'
    | key b < key b' -> singleton b `merge` singleton b'
    | key b == key b' -> singleton b
    | otherwise -> singleton b' `merge` singleton b
  M tl tr
    | key b <= maxKey tl -> insert b tl `merge` tr
    | otherwise -> tl `merge` insert b tr
    
delete :: (Ord k, Ord p) => k -> PSQ k p -> PSQ k p
delete k psq = case treeView psq of
  Z -> Void
  S b
    | k == key b -> Void
    | otherwise -> singleton b
  M tl tr
    | k <= maxKey tl -> delete k tl `merge` tr
    | otherwise -> tl `merge` delete k tr

### A Balanced Scheme

Priority search queues can be augmented with a balancing scheme to improve the situation with operations such as `insert` which otherwise could result in degenerate trees.

For the scheme we will use, a tree is weight-balanced if for all nodes either both subtrees have at most one element or one subtree does not have more than `w` times as many elements as it's sibling, where `w` is some constant > 3.75. To check and maintain this invariant, each node in a loser tree has it's size cached on it's root node.

In [5]:
w :: Int
w = 4

balance :: (Ord k, Ord p) => (k, p) -> LTree k p -> k -> LTree k p -> LTree k p
balance b l k r
  | size l + size r < 2 = node b l k r
  | size r > w * size l = balanceLeft b l k r
  | size l > w * size r = balanceRight b l k r
  | otherwise           = node b l k r

balanceLeft :: (Ord k, Ord p) => (k, p) -> LTree k p -> k -> LTree k p -> LTree k p
balanceLeft b l k r@(Loser _ _ rl _ rr)
  | size rl < size rr = singleLeft b l k r
  | otherwise         = doubleLeft b l k r

balanceRight :: (Ord k, Ord p) => (k, p) -> LTree k p -> k -> LTree k p -> LTree k p
balanceRight b l@(Loser _ _ ll _ lr) k r
  | size lr < size ll = singleRight b l k r
  | otherwise         = doubleRight b l k r
  
singleLeft :: (Ord k, Ord p) => (k, p) -> LTree k p -> k -> LTree k p -> LTree k p
singleLeft b1 t1 k1 (Loser _ b2 t2 k2 t3)
  | key b2 <= k2 && prio b1 <= prio b2
  = node b1 (node b2 t1 k1 t2) k2 t3
  | otherwise = node b2 (node b1 t1 k1 t2) k2 t3

singleRight :: (Ord k, Ord p) => (k, p) -> LTree k p -> k -> LTree k p -> LTree k p
singleRight b1 (Loser _ b2 t1 k1 t2) k2 t3
  | key b2 > k1 && prio b1 <= prio b2
  = node b1 t2 k1 (node b2 t2 k2 t3)
  | otherwise = node b2 t1 k1 (node b1 t2 k2 t3)
  
doubleLeft :: (Ord k, Ord p) => (k, p) -> LTree k p -> k -> LTree k p -> LTree k p
doubleLeft b1 t1 k1 (Loser _ b2 t2 k2 t3)
  = singleLeft b1 t1 k1 (singleRight b2 t2 k2 t3)

doubleRight :: (Ord k, Ord p) => (k, p) -> LTree k p -> k -> LTree k p -> LTree k p
doubleRight b1 (Loser _ b2 t1 k1 t2) k2 t3
  = singleRight b1 (singleLeft b2 t1 k1 t2) k2 t3

To adapt the dictionary operations to use balancing, occurences of `Loser` should be replaced with `balance`. If the operation does not change the shape of the tree (such as `adjust`), then `node` can be used.